# **Tarea 08.- Uso de mallador `gmsh` con GUI y Phyton**
---
## *Subcoordinación de Posgrado y Educación Continua.*
### [Instituto Mexicano de Tecnología del Agua](https://www.gob.mx/imta).<br>

<img src="Imagenes/imta_logo.png" style="height: 5em; vertical-align: middle;">
<img src="Imagenes/Mexico.jpg" style="height: 5em; vertical-align: middle;">

**Alumno: Ing. Omar Ulises Robles Pereyra** <br>
**Docente: Dr. Gabriel Ruiz Martinez** <br>

[![Open In Colab](Imagenes/colab-badge.svg)](https://colab.research.google.com/github/OmarURP/Tratamiento_Datos/blob/master/Clase_08.ipynb)

---

#### **Basado en los apuntes del Dr. Gabriel Ruiz Martinez**

**Descripción de la actividad:**
- Generar un documento con un procedimiento detallado donde se explique cómo generar una malla para OpenFoam, utilizando GMSH, python y su interfaz gráfica, considerando los siguientes pasos. <br>

**Objetivos de aprendizaje**
- Comprender el proceso de generación de una malla para OpenFoam usando GMSH.
- Desarrollar habilidades en el uso de la interfaz gráfica de GMSH para crear geometrías y mallas.
- Aprenda a explicar la importancia de cada paso en el proceso de generación de mallas.
- Adquirir experiencia en la documentación de procedimientos técnicos de manera clara y efectiva.
- Mejorar la capacidad de usar ayudas visuales, como capturas de pantalla, para respaldar las explicaciones en la documentación técnica.

**Referencias:**<br>
- https://gmsh.info/doc/texinfo/gmsh.html#Gmsh-application-programming-interface<br>
- Moukalled, F., Mangani, L., & Darwish, M. (2016). *The finite volume method in computational fluid dynamics: An advanced introduction with OpenFOAM and Matlab.* Springer. https://doi.org/10.1007/978-3-319-16874-6
---

In [1]:
import gmsh

## **Comparando cada paso en la GUI de GMSH con su equivalente en Python.**
#### **1. Inicializando la interfaz**
<img src="Datos/08_Tarea/1.png" style="height: 20em; vertical-align: middle;">

In [2]:
# Inicializa la API de GMSH - debe llamarse antes de cualquier otra función de GMSH
gmsh.initialize()

# Habilita el output en la terminal (1 activado, 0 desactivado)
gmsh.option.setNumber("General.Terminal", 1)

# Resolución característica del mesh - controla el tamaño de los elementos
# Valores más pequeños generan mallas más finas
resol = 0.01

---
#### **2. Crear puntos**
Geometría → Entidades elementales → Agregar → Puntos <br>
- Click en el canvas para posicionar cada punto
- Ingresar coordenadas (x, y, z) manualmente
- Definir tamaño característico de malla <br>

<img src="Datos/08_Tarea/2.png" style="height: 20em; vertical-align: middle;">

In [3]:
# Crea puntos en el espacio 3D que serán los vértices de la geometría
# gmsh.model.geo.addPoint(x, y, z, meshSize, tag)
# Los últimos 4 parámetros: (x, y, z) coordenadas, meshSize para refinamiento local, tag identificador

# Punto 1: esquina inferior izquierda (origen)
gmsh.model.geo.addPoint(0.0, 0.0, 0.0, resol, 1)
# Punto 2: esquina inferior derecha (10 cm en x)
gmsh.model.geo.addPoint(0.1, 0.0, 0.0, resol, 2)
# Punto 3: esquina superior derecha (10 cm x 10 cm)
gmsh.model.geo.addPoint(0.1, 0.1, 0.0, resol, 3)
# Punto 4: esquina superior izquierda (10 cm en y)
gmsh.model.geo.addPoint(0.0, 0.1, 0.0, resol, 4)

4

---
#### **3. Crear líneas**
Geometría → Entidades elementales → Agregar → Línea
- Seleccionar punto inicial
- Seleccionar punto final
- Presionar `e` para finalizar<br>

<img src="Datos/08_Tarea/3.png" style="height: 20em; vertical-align: middle;">

In [4]:
# Crea líneas rectas conectando dos puntos
# gmsh.model.geo.addLine(startPoint, endPoint, tag)
# Estas líneas formarán los bordes de la superficie cuadrada

# Línea 1: borde inferior (punto 1 → punto 2)
gmsh.model.geo.addLine(1, 2, 1)
# Línea 2: borde derecho (punto 2 → punto 3)
gmsh.model.geo.addLine(2, 3, 2)
# Línea 3: borde superior (punto 3 → punto 4) - esta será la pared móvil
gmsh.model.geo.addLine(3, 4, 3)
# Línea 4: borde izquierdo (punto 4 → punto 1)
gmsh.model.geo.addLine(4, 1, 4)

4

---
#### **4. Crear superficie plana**
Geometría → Entidades elementales → Añadir → Superficie plana <br>
- Seleccionar las líneas que forman el contorno cerrado
- Presionar `e` para crear la superficie
- GMSH automáticamente crea el Curve Loop

<img src="Datos/08_Tarea/4.png" style="height: 20em; vertical-align: middle;">

In [5]:
# Crea un loop cerrado de curvas que define el perímetro de una superficie
# gmsh.model.geo.addCurveLoop(curveTags, tag)
# Las curvas deben formar un contorno cerrado y continuo
# El orden de las líneas debe ser consistente (todas en sentido horario o antihorario)

gmsh.model.geo.addCurveLoop([1, 2, 3, 4], 1)

1

In [6]:
# Crea una superficie plana definida por uno o más curve loops
# gmsh.model.geo.addPlaneSurface(wireTags, tag)
# El primer loop es el contorno exterior, los siguientes serían huecos internos
# Esta superficie será la base para la extrusión 3D

gmsh.model.geo.addPlaneSurface([1], 1)

1

---
#### **5. Mallado transfinito - Curvas**
Malla → Definir → Transfinito → Curva
- Click en "Seleccionar curvas"
- Seleccionar las líneas (1, 2, 3, 4)
- Ingresar número de nodos: 20
- Tipo de progresión: 1 (uniforme)
- Click "Añadir"<br>

<img src="Datos/08_Tarea/5.png" style="height: 20em; vertical-align: middle;">

In [7]:
# Aplica mallado transfinito a una curva = distribución estructurada de nodos
# gmsh.model.geo.mesh.setTransfiniteCurve(tag, numNodes, meshType, coef)
# numNodes: número de nodos en la curva (20 nodos = 19 elementos)
# coef: factor de progresión geométrica (1 = espaciado uniforme)

# Línea 3: borde superior - 20 nodos uniformemente espaciados
gmsh.model.geo.mesh.setTransfiniteCurve(3, 20, coef=1)
# Línea 4: borde izquierdo - 20 nodos uniformemente espaciados
gmsh.model.geo.mesh.setTransfiniteCurve(4, 20, coef=1)
# Línea 1: borde inferior - 20 nodos uniformemente espaciados
gmsh.model.geo.mesh.setTransfiniteCurve(1, 20, coef=1)
# Línea 2: borde derecho - 20 nodos uniformemente espaciados
gmsh.model.geo.mesh.setTransfiniteCurve(2, 20, coef=1)

---
#### **6. Mallado Transfinito - Superficie**
Malla → Definir → Transfinito → Superficie
- Seleccionar la superficie creada
- Elegir "Arrangement": Left (por defecto)
- Click "Añadir"<br>

<img src="Datos/08_Tarea/6.png" style="height: 20em; vertical-align: middle;"><br>

In [8]:
# Aplica mallado transfinito a una superficie = malla estructurada
# gmsh.model.geo.mesh.setTransfiniteSurface(tag, arrangement, cornerTags)
# Genera una malla cuadrilátera estructurada cuando todas las curvas del borde son transfinitas
# arrangement especifica cómo se organizan los elementos (por defecto "Left")

gmsh.model.geo.mesh.setTransfiniteSurface(1)

Malla → 2D
- Genera la malla en todas las superficies
- Ver resultados: Tools → Statistics o presionar `Ctrl`+`I`<br>

<img src="Datos/08_Tarea/7.png" style="height: 20em; vertical-align: middle;"><br>

---
#### **7. Recombinar elementos**
Malla → Definir → Recombinar
Seleccionar la superficie (tag = 1)
Esto convierte triángulos → cuadriláteros
Sin esto, la extrusión podría generar prismas/tetraedros en lugar de hexaedros<br>

<img src="Datos/08_Tarea/8.png" style="height: 20em; vertical-align: middle;"><br>

In [9]:
# Recombina elementos triangulares en cuadriláteros (2D) o tetraedros en hexaedros (3D)
# gmsh.model.geo.mesh.setRecombine(dim, tag)
# dim: dimensión de la entidad (2 = superficie, 3 = volumen)
# tag: identificador de la superficie a recombinar
# Esencial para generar mallas hexaédricas en OpenFOAM

gmsh.model.geo.mesh.setRecombine(2,1)

---
#### **8. Extrusión (Creación de Volumen)**
Geometría → Entidades elementales → Extrusión → Traslación
- Seleccionar la superficie (tag = 1)
- Ingresar vector de traslación:
    - dx = 0
    - dy = 0
    - dz = 0.01 (1 cm en dirección z)
- Capas: 1
Recombinar: Activado para generar hexaedros<br>

<img src="Datos/08_Tarea/9.png" style="height: 20em; vertical-align: middle;"><br>

In [10]:
# Extruye entidades geométricas en la dirección especificada
# gmsh.model.geo.extrude(dimTags, dx, dy, dz, numElements, heights, recombine)
# dimTags: lista de tuplas (dimensión, tag) de entidades a extruir
# (dx, dy, dz): vector de extrusión (0, 0, 0.01) = 1 cm en dirección z
# numElements: [1] = número de capas de elementos en la extrusión
# recombine=True: genera hexaedros en lugar de tetraedros/prismas
# Retorna lista de entidades creadas: volumen, superficies laterales, etc.

gmsh.model.geo.extrude([(2,1)], 0, 0, 0.01, [1], recombine=True)

[(2, 26), (3, 1), (2, 13), (2, 17), (2, 21), (2, 25)]

In [ ]:
# Sincroniza el modelo CAD con el modelo interno de GMSH
# gmsh.model.geo.synchronize()
# Debe llamarse después de modificar la geometría y antes de:
#   - Generar la malla
#   - Crear grupos físicos
#   - Aplicar operaciones de mallado
# Transfiere todas las entidades del kernel de geometría al modelo de malla

gmsh.model.geo.synchronize()

---
#### **9. Añadir grupos físicos - Superficies**
Geometría → Grupos físicos → Añadir → Superficie
Para cada grupo físico:
a) Moving Wall (pared superior)
Nombre: movingWall
Tag físico: 27
Seleccionar superficie 13 (top extruded surface)

b) Fixed Walls (paredes fijas)
Nombre: fixedWalls
Tag físico: 28
Seleccionar superficies: 17, 21, 25 (laterales)

c) Front and Back (caras 2D)
Nombre: frontandback
Tag físico: 29
Seleccionar superficies: 1 (bottom), 26 (top)<br>

<img src="Datos/08_Tarea/10.png" style="height: 20em; vertical-align: middle;"><br>

In [12]:
# Crea grupos físicos = etiquetas para condiciones de frontera en OpenFOAM
# gmsh.model.addPhysicalGroup(dim, tags, tag, name)
# dim: dimensión (1=línea, 2=superficie, 3=volumen)
# tags: lista de IDs de entidades a incluir en el grupo
# tag: identificador numérico del grupo físico
# name: nombre descriptivo para identificar en OpenFOAM

# Superficie 13: pared superior móvil (cavity)
gmsh.model.addPhysicalGroup(2, [13], 27, "movingWall")
# Superficies 17, 21, 25: paredes fijas laterales e inferior
gmsh.model.addPhysicalGroup(2, [17, 21, 25], 28, "fixedWalls")
# Superficies 1 y 26: caras frontal y trasera (empty en OpenFOAM 2D)
gmsh.model.addPhysicalGroup(2, [1, 26], 29, "frontandback")
# Línea 1: grupo físico de línea (no necesario para OpenFOAM 3D)
# Puede causar advertencias pero no afecta la conversión
# gmsh.model.addPhysicalGroup(1, [1], 30, "water")

29

---
#### **10. Añadir grupos físicos - Volumen**
Nombre: internal
Tag físico: 1
Seleccionar el volumen 3D (tag = 1)<br>

<img src="Datos/08_Tarea/11.png" style="height: 20em; vertical-align: middle;"><br>

In [13]:
# Define el grupo físico para el VOLUMEN 3D
# gmsh.model.addPhysicalGroup(3, [1], 1, "internal")
# Sin esto, gmshToFoam NO encuentra elementos 3D y falla
# Volumen 1: región interna del dominio computacional
# OpenFOAM necesita al menos un physical group 3D para importar celdas

gmsh.model.addPhysicalGroup(3, [1], 1, "internal")

1

---
#### **11. Generación de malla 2D y 3D**
`Malla → 3D`<br>

Genera elementos volumétricos (hexaedros)<br>

Verifica con Tools → Statistics:
- Hexahedra: 361 
- Tetrahedra: 0<br>

<img src="Datos/08_Tarea/12.png" style="height: 20em; vertical-align: middle;"><br>

In [14]:
# Genera la malla para entidades de dimensión <= dim
# gmsh.model.mesh.generate(dim)
# dim=1: malla 1D (solo curvas)
# dim=2: malla 2D (curvas + superficies)
# dim=3: malla 3D completa (curvas + superficies + volúmenes)
# Aplica todos los algoritmos de mallado especificados (transfinito, recombine, etc.)

gmsh.model.mesh.generate(3)

---
#### **12. Guardar el archivo**
`gmsh → Guardar`<br>
Nombre: cavity2_2.msh<br>
Formato: MSH2 ASCII (versión 2.2)<br>

File → Export → Format → Version 2 ASCII<br>

<img src="Datos/08_Tarea/13.png" style="height: 20em; vertical-align: middle;"><br>

In [15]:
# Configura la versión del formato de archivo .msh
# gmsh.option.setNumber("Mesh.MshFileVersion", version)
# Versión 2.2: formato ASCII compatible con utilidad de OpenFOAM (gmshToFoam)
gmsh.option.setNumber("Mesh.MshFileVersion", 2.2)

In [16]:
# Exporta la malla al archivo especificado
# gmsh.write(filename)
# Formato determinado por la extensión (.msh, .vtk, .stl, etc.)
# Incluye geometría, malla y grupos físicos
ruta = "./Resultados/gmsh"
gmsh.write(ruta + "/cavity2_2.msh")

#### **Comparación de archhivos `.msh` GUI vs Python**
<img src="Datos/08_Tarea/14.png" style="height: 20em; vertical-align: middle;"><br>

---
#### **13. Visualización de gmsh Python**

In [17]:
# Lanza la interfaz gráfica (GUI) de gmsh
# gmsh.fltk.run()
# Permite visualizar la geometría y la malla generada
# Útil para verificación visual
# Bloquea la ejecución hasta que se cierre la ventana

gmsh.fltk.run()

<img src="Datos/08_Tarea/15.png" style="height: 20em; vertical-align: middle;"><br>

In [18]:
# Finaliza la API de GMSH y libera recursos
# gmsh.finalize()
# Cierra archivos abiertos y limpia memoria

gmsh.finalize()

---
#### **14. Conversión a OpenFOAM**
Los archívos mínimos con que se debe contar antes de antes de correr la utilidad `gmshToFoam` son:<br>

Una vez que se cuenta con la estructura anterior, en el bash se ejecuta la utilidad `gmshToFoam`

<img src="Datos/08_Tarea/16.png" style="height: 30em; vertical-align: middle;"><br>
La imagen muestra la ejecución del comando gmshToFoam cavity2_2.msh en el entorno de OpenFOAM, donde se realiza la conversión de la malla generada en GMSH (formato .msh versión 2.2) al 
formato nativo de OpenFOAM (polyMesh). <br>
Esta configuración confirma que se obtuvo una malla completamente hexaédrica y estructurada, ideal para simulaciones CFD de alta precisión (2016, F. Moukalled, L. Magani, M.Darwish). La malla 100% hexaédrica proporciona:
- Mayor precisión numérica en las soluciones
- Mejor convergencia de los métodos iterativos
- Reducción del error de discretización
- Ortogonalidad perfecta (non-orthogonality = 0)<br>

<img src="Datos/08_Tarea/17.png" style="height: 30em; vertical-align: middle;"><br>

Una vez que se ha generado la malla, se puede ver que ahora se cuenta con una nueva carpeta llamada `polyMesh` en la carpeta `constant` que contiene la siguiente estructura.

---
#### **15. Correr la malla en OpenFoam**
Para correr el modelo que  se ha mallado, es necesario realizar cambios en el archivo de la carpeta `constant/boundary`, debido a que `gmshToFoam` no sabe qué tipo de condición de frontera (tipo base o numérica) aplicar a cada superficie, solo reconoce que son "fronteras" y les asigna el tipo genérico patch. Por ende, el usuario debes especificar el comportamiento físico correcto para cada frontera según el problema.

##### **Significado de condiciones de frontera de tutorial `cavity`**
**Significado de empty:**
- No se resuelven ecuaciones en caras empty
- Reduce problema 3D → 2D
- Gradientes normales = 0 automáticamente<br>

**Significado de wall:**
- Activa tratamiento especial de pared
- Calcula esfuerzos cortantes (wall shear stress)
- Aplica funciones de pared en turbulencia
- Permite usar condición tipo noSlip en 0/U
---
#### **Creación de la carpeta `0/`**
La carpeta `0/` contiene las condiciones iniciales y de frontera de todas las variables del problema (velocidad, presión, temperatura, turbulencia, etc.) en el tiempo `t = 0`.

En estos archivos se debe tener cuidado en tener la misma escritura y respetar en todo momento las entradas de condiciones de frontera tipo base y numéricas almacenadas en el `foamDictionary` de OpenFOAM.
#### **1. Archivo `U` (Velocidad)**

#### **2. Archivo `p` (Presión)**

#### **3. Archivo `k` (Energía Cinética Turbulenta)**

#### **4. Archivo `epsilon` (Disipación Turbulenta)**

#### **5. Archivo `omega` (Disipación Específica)**

#### **6. Archivo `nut` (Viscosidad Turbulenta)**

#### **7. Archivo `nuTilda` (Variable Spalart-Allmaras)**

Una vez se configuró la carpeta 0, se cuenta con todos los requerimientos para realizar la simulación del caso en estudio.<br>
Solo resta cruzar los dedos y digitar en el bash `foamRun`

<img src="Datos/08_Tarea/18.png" style="height: 35em; vertical-align: middle;"><br>